In [13]:
import pandas as pd
import json
import requests
import copy
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold
from sklearn import model_selection, preprocessing, metrics
import seaborn as sns
import  estateFunction

In [14]:
data2018=pd.read_csv('dataAfterPreprocess/data2018') 
data2017=pd.read_csv('dataAfterPreprocess/data2017')
data2016=pd.read_csv('dataAfterPreprocess/data2016') 
data2015=pd.read_csv('dataAfterPreprocess/data2015') 
data2014=pd.read_csv('dataAfterPreprocess/data2014') 
data2013=pd.read_csv('dataAfterPreprocess/data2013') 
data2012=pd.read_csv('dataAfterPreprocess/data2012') 
data2011=pd.read_csv('dataAfterPreprocess/data2011') 
data2010=pd.read_csv('dataAfterPreprocess/data2010') 

In [15]:
###merge all data
train=pd.concat([data2018,data2017,data2016,data2015,data2014,data2013,data2012,data2011,data2010],axis=0)
train=train.reset_index()
del train['index']
train.shape

(710731, 16)

In [16]:
###checikng the null data
train.isnull().sum()

시군구             0
번지              0
본번              0
부번              0
단지명             0
전용면적(㎡)         0
계약년월            0
거래금액(만원)        0
층               0
건축년도            0
도로명          1345
latitude        0
longitude       0
big             0
small           0
month           0
dtype: int64

In [17]:
###결측치 drop함
train=train.drop(train.index[train['도로명'].isnull()],0)

In [18]:
train.isnull().sum().sum(),len(train)

(0, 709386)

In [ ]:
###이상치를제거하자

In [19]:
###280이상은 제거하자.
train.sort_values(["전용면적(㎡)"],ascending=False)[:20]

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,거래금액(만원),층,건축년도,도로명,latitude,longitude,big,small,month
3468,서울특별시 강남구 도곡동,193-1,193,1,힐데스하임빌라,424.320,201604,498000.0,10,1998,남부순환로377길,37.488126,127.045025,강남구,도곡동,4
266300,서울특별시 서초구 서초동,1518-2,1518,2,월드빌라트,395.060,201809,290000.0,10,1996,명달로16길,37.485489,127.005450,서초구,서초동,9
506741,서울특별시 강남구 청담동,89-12,89,12,청담동마크노빌,325.390,201302,420000.0,6,2008,도산대로59길,37.525378,127.042161,강남구,청담동,2
405833,서울특별시 용산구 이촌동,302-69,302,69,정우,317.360,201509,118000.0,7,1972,이촌로54길,37.520833,126.967177,용산구,이촌동,9
287806,서울특별시 용산구 이촌동,302-69,302,69,정우,317.360,201810,157000.0,5,1972,이촌로54길,37.520833,126.967177,용산구,이촌동,10
405832,서울특별시 용산구 이촌동,302-69,302,69,정우,317.360,201504,85000.0,2,1972,이촌로54길,37.520833,126.967177,용산구,이촌동,4
543742,서울특별시 서초구 반포동,612-33,612,33,펠리체,283.760,201302,180000.0,5,2007,동광로27길,37.496575,126.994165,서초구,반포동,2
264798,서울특별시 서초구 반포동,612-33,612,33,펠리체,283.760,201808,185000.0,5,2007,동광로27길,37.496575,126.994165,서초구,반포동,8
507711,서울특별시 강동구 길동,351-2,351,2,신세대빌라트,283.000,201312,63000.0,9,1999,천중로,37.540564,127.144661,강동구,길동,12
106928,서울특별시 중구 흥인동,13-1,13,1,청계천 두산위브더제니스,273.967,201607,228977.0,28,2014,다산로46길,37.566863,127.017110,중구,흥인동,7


In [20]:
##삭제됨
train=train.drop(train.index[train["전용면적(㎡)"]>280],0)

In [ ]:
###거래금액은 이상치로 판단이 잘안됨.
###다음의 거래를 투기로보고
###1.3개월에 2번 거래된거 
###2.전판매가 5배이상 제거하자

In [23]:
###단지명,층,전용면적까지 같으면 같은건물로 보고
###계약년월 순대로 정렬함
train=train.sort_values(["단지명","층","전용면적(㎡)","계약년월"])

In [24]:
train=train.reset_index()
del train['index']

In [ ]:
#3개월에 2번 거래된것 삭제
#전거래와 5배 차이 나는 거래들 삭제
train=estateFunction.deleteOutlier(train)
train=estateFunction.deleteOutlier2(train)

In [ ]:
train.to_csv("dataAfterPreprocess/alltrain",header=True,index=False)

In [26]:
train=pd.read_csv("dataAfterPreprocess/alltrain")

In [27]:
#대략 20만개 정도 사라짐
len(train)

555855

In [28]:
#비교를 위해서 공시가격데이터를 처리하자
public=pd.read_csv('publicData/public2018.csv',encoding="cp949") 

C:\Users\ju\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,6,11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
public[:5]

,기준연도,기준월,법정동코드,도로명주소,시도,시군구,읍면,동리,특수지코드,본번,부번,특수지명,단지명,동명,호명,전용면적,공시가격
0,2018,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,1,111,187.49,723000000.0
1,2018,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,1,112,187.49,723000000.0
2,2018,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,1,121,157.25,702000000.0
3,2018,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,1,122,157.25,702000000.0
4,2018,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,1,131,152.33,698000000.0


In [30]:
###public 에서 서울특별시만뽑음
def filterSeoul(x):
    return '서울특별시' in x 
public=public.loc[public["도로명주소"].apply(filterSeoul)]

In [31]:
len(public)

2444451

In [32]:
###단지명,전용면적,층 같은 건물은 공시가격 평균낼거임
###먼저 층수 파악이안되는 데이터 버림
public=public.drop(public.index[public["호명"].apply(str).apply(len)>4])  

In [33]:
def makingLayer(x):
    if len(x)==4:
        return x[0]+x[1]
    elif len(x)==3:
        return x[0]

In [34]:
###호수로 층 파악하고 가격 평균냈음
public['층']=public['호명'].apply(str).apply(makingLayer)
grouped=public['공시가격'].groupby([public['단지명'], public['전용면적'], public['층']])
public=pd.DataFrame(grouped.mean()).reset_index()

In [35]:
public

,단지명,전용면적,층,공시가격
0,경성빌라(179-6),75.45,1,250000000.0
1,경성빌라(179-6),75.45,2,264000000.0
2,경성빌라(179-6),75.45,3,258000000.0
3,경성빌라(179-6),79.69,1,263000000.0
4,경성빌라(179-6),79.69,2,277000000.0
...,...,...,...,...
908638,參元하우스,43.35,2,174000000.0
908639,參元하우스,43.35,3,174000000.0
908640,參元하우스,55.53,4,224000000.0
908641,參元하우스,59.74,4,113000000.0


In [37]:
###원래의 데이터를 train과 test로 나눈어서 test에는 비교를 할 수 있도록 공시가격을 넣어주자
###train test로 나눔
finaltest=train.loc[train["계약년월"]>201811].reset_index()
del finaltest['index']
finaltrain=train.loc[train["계약년월"]<=201811].reset_index()
del finaltrain['index']
finaltest['공시가격']=0
###public의 층에 문자열있어서 이거를 바꾸어주어야지 비교가가능함
finaltest['층']=finaltest['층'].apply(str)

In [23]:
finaltest[:3]

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,거래금액(만원),층,건축년도,도로명,latitude,longitude,big,small,month,공시가격
0,서울특별시 영등포구 영등포동1가,145,145,0,(145-0),14.24,201812,12500.0,5,2012,경인로114길,37.517201,126.912720,영등포구,영등포동1가,12,0
1,서울특별시 마포구 성산동,650,650,0,(650-0),84.97,201812,57500.0,8,2006,월드컵로26길,37.559705,126.910651,마포구,성산동,12,0
2,서울특별시 구로구 구로동,745-57,745,57,(745-57),35.46,201812,10000.0,5,1997,구로동로12길,37.487121,126.885867,구로구,구로동,12,0


In [24]:
public[:3]

,단지명,전용면적,층,공시가격
0,경성빌라(179-6),75.45,1,250000000.0
1,경성빌라(179-6),75.45,2,264000000.0
2,경성빌라(179-6),75.45,3,258000000.0


In [26]:
######공시데이터 중에서 단지명 전용면적 층이 테스트 데이터와 같으면 공시가격 넣겠다
for i in range(len(public)):
    finaltest['공시가격'].loc[finaltest.index[(public['단지명'][i]==finaltest['단지명'])&(public['전용면적'][i]
    ==finaltest['전용면적(㎡)'])&(public['층'][i]==finaltest['층'])]]=public['공시가격'][i]

C:\Users\ju\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [28]:
###1371개중 1192개가 일치함
len(finaltest.loc[finaltest['공시가격']>0]),len(finaltest)

(1192, 1371)

In [29]:
finaltest=finaltest.loc[finaltest['공시가격']>0]
###단위가 다르므로 공시가격을 (만원)단위로 만들겠음
finaltest['공시가격(만원)']=finaltest['공시가격']/10000

C:\Users\ju\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
###2010/01 부터 2018/11 까지
finaltrain.to_csv("dataAfterPreprocess/finaltrain",header=True,index=False)
###2018/12 데이터인데 공시가격 넣어둠
finaltest.to_csv("dataAfterPreprocess/finaltest",header=True,index=False)

In [31]:
finaltrain

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,거래금액(만원),층,건축년도,도로명,latitude,longitude,big,small,month
0,서울특별시 성동구 행당동,1-10,1,10,(1-10),75.70,201411,15000.0,4,2003,마조로1길,37.561560,127.039184,성동구,행당동,11
1,서울특별시 종로구 명륜3가,1-102,1,102,(1-102),69.54,201405,17000.0,-1,1997,명륜2길,37.589464,126.995258,종로구,명륜3가,5
2,서울특별시 종로구 명륜3가,1-102,1,102,(1-102),70.05,201607,19000.0,1,1997,명륜2길,37.589472,126.995254,종로구,명륜3가,7
3,서울특별시 종로구 명륜3가,1-102,1,102,(1-102),70.05,201611,19000.0,1,1997,명륜2길,37.589472,126.995254,종로구,명륜3가,11
4,서울특별시 종로구 명륜3가,1-102,1,102,(1-102),59.97,201405,17000.0,2,1997,명륜2길,37.589464,126.995258,종로구,명륜3가,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554479,서울특별시 서초구 반포동,577-79,577,79,힐하우스(577-79),88.35,201802,59000.0,1,2003,동광로46길,37.493616,127.001104,서초구,반포동,2
554480,서울특별시 서초구 방배동,802-3,802,3,힐하우스(802-3),212.48,201803,87000.0,1,2003,방배로34길,37.493295,126.995481,서초구,방배동,3
554481,서울특별시 서초구 방배동,802-3,802,3,힐하우스(802-3),212.48,201003,90000.0,3,2003,방배로34길,37.493295,126.995481,서초구,방배동,3
554482,서울특별시 서초구 방배동,802-3,802,3,힐하우스(802-3),195.82,201003,84000.0,4,2003,방배로34길,37.493295,126.995481,서초구,방배동,3


In [32]:
finaltest

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,거래금액(만원),층,건축년도,도로명,latitude,longitude,big,small,month,공시가격,공시가격(만원)
0,서울특별시 영등포구 영등포동1가,145,145,0,(145-0),14.24,201812,12500.0,5,2012,경인로114길,37.517201,126.912720,영등포구,영등포동1가,12,78000000.0,7800.0
1,서울특별시 마포구 성산동,650,650,0,(650-0),84.97,201812,57500.0,8,2006,월드컵로26길,37.559705,126.910651,마포구,성산동,12,345000000.0,34500.0
2,서울특별시 구로구 구로동,745-57,745,57,(745-57),35.46,201812,10000.0,5,1997,구로동로12길,37.487121,126.885867,구로구,구로동,12,66000000.0,6600.0
3,서울특별시 구로구 구로동,771-2,771,2,(771-2),58.11,201812,17000.0,3,1996,구로동로20길,37.487681,126.888001,구로구,구로동,12,121000000.0,12100.0
4,서울특별시 구로구 구로동,792-78,792,78,(792-78),45.69,201812,9800.0,1,1996,디지털로27라길,37.486474,126.888500,구로구,구로동,12,74000000.0,7400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366,서울특별시 성북구 성북동1가,35-22,35,22,휴아름 스튜디오,16.53,201812,20200.0,14,2015,창경궁로,37.588437,127.005417,성북구,성북동1가,12,81000000.0,8100.0
1367,서울특별시 동작구 흑석동,336,336,0,흑석한강푸르지오,127.52,201812,154500.0,5,2012,흑석한강로,37.503212,126.965443,동작구,흑석동,12,761000000.0,76100.0
1368,서울특별시 동작구 흑석동,336,336,0,흑석한강푸르지오,127.52,201812,159500.0,11,2012,흑석한강로,37.503212,126.965443,동작구,흑석동,12,786500000.0,78650.0
1369,서울특별시 강서구 화곡동,1011-6,1011,6,희훈리치파크,115.50,201812,59500.0,11,2002,강서로45다길,37.546381,126.834013,강서구,화곡동,12,351000000.0,35100.0
